# STEP 6: Repeat the computation from the facts & dimension table

Note: You will not have to write any code in this notebook. It's purely to illustrate the performance difference between Star and 3NF schemas.

Start by running the code in the cell below to connect to the database.

In [ ]:
%%capture
# ^hide output of this cell (comment to debug)

# run step 5 (and 4) to create and populate tables (can take a couple of seconds)
%run "E1 - Step 5.ipynb"

# retrieve tables from step 4 & 5
%store -r tables

from src.database import (
    get_pg_connection,
    execute_pg_query,
    create_pg_table,
    insert_pg_rows,
    drop_pg_table,
    close_pg_connection,
)
from IPython.display import display
import pandas as pd

# get active connection
conn = get_pg_connection("pagila")

## 6.1 Facts Table has all the needed dimensions, no need for deep joins

In [ ]:
result = execute_pg_query(
    conn,
    """
SELECT movie_key, date_key, customer_key, sales_amount
FROM factSales 
limit 5;
""",
)
display(result.style.set_caption("factSales").hide(axis="index"))

## 6.2 Join fact table with dimensions to replace keys with attributes

As you run each cell, pay attention to the time that is printed. Which schema do you think will run faster?

##### Star Schema

In [ ]:
%%time
result = execute_pg_query(
    conn,
    """
SELECT dimMovie.title, dimDate.month, dimCustomer.city, sum(sales_amount) as revenue
FROM factSales 
JOIN dimMovie    on (dimMovie.movie_key      = factSales.movie_key)
JOIN dimDate     on (dimDate.date_key         = factSales.date_key)
JOIN dimCustomer on (dimCustomer.customer_key = factSales.customer_key)
group by (dimMovie.title, dimDate.month, dimCustomer.city)
order by dimMovie.title, dimDate.month, dimCustomer.city, revenue desc;
""",
)

##### 3NF Schema

In [ ]:
%%time
result = execute_pg_query(
    conn,
    """
SELECT f.title, EXTRACT(month FROM p.payment_date) as month, ci.city, sum(p.amount) as revenue
FROM payment p
JOIN rental r    ON ( p.rental_id = r.rental_id )
JOIN inventory i ON ( r.inventory_id = i.inventory_id )
JOIN film f ON ( i.film_id = f.film_id)
JOIN customer c  ON ( p.customer_id = c.customer_id )
JOIN address a ON ( c.address_id = a.address_id )
JOIN city ci ON ( a.city_id = ci.city_id )
group by (f.title, month, ci.city)
order by f.title, month, ci.city, revenue desc;
""",
)

# Conclusion

We were able to show that:
* The star schema is easier to understand and write queries against.
* Queries with a star schema are more performant.

Drop all tables and close the connection to the database.

In [ ]:
for table in tables:
    drop_pg_table(conn, table, cascade=True)
close_pg_connection(conn)